### Cálculo perigosidade Incêndio Florestal

**Variáveis:**

- Elevation
- Slope
- Aspect
- LULC

In [ ]:
burn_areas = '/mnt/x/atrisk_fire1/aa24/aa_1975_2021_v2.shp'

burn_year = '/mnt/x/atrisk_fire1/aa2024_prob'

nyears = 2021 - 1979

dem = '/mnt/x/atrisk_fire1/topografia/dem_pnse.tif'

lulc_maps = {
    '/mnt/x/atrisk_fire1/lulc/r_cos1995.tif' : 12,
    '/mnt/x/atrisk_fire1/lulc/r_cos2007.tif' : 3,
    '/mnt/x/atrisk_fire1/lulc/r_cos2010.tif' : 5,
    '/mnt/x/atrisk_fire1/lulc/r_cos2015.tif' : 3,
    '/mnt/x/atrisk_fire1/lulc/r_cos2018.tif' : 4,
}

outrst = '/home/jasp/firerisk1/fire_pnse_perigo.tif'

### Python packages

In [ ]:
import os

from glass.cons.firerisk import ELEVATION_RULES, SLOPE_RULES, ASPECT_RULES
from glass.pys.oss import mkdir
from glass.rst.surf import slope_aspect
from glass.rst.rcls import rcls_rasters
from glass.ete.lri import grass_lri
from glass.rst.stats import count_region_in_shape
from glass.rst.alg import rstcalc

### Workspace

In [ ]:
ws = os.path.dirname(outrst)

surf_folder = mkdir(os.path.join(ws, 'surface'), overwrite=True)
vars_folder = mkdir(os.path.join(ws, 'vars'), overwrite=True)

vars = {'elevation' : dem}
vrules = {
    'elevation' : ELEVATION_RULES,
    'slope'     : SLOPE_RULES,
    'aspect'    : ASPECT_RULES
}

### Generate Slope and Aspect

In [ ]:
vars['slope'], vars['aspect'] = slope_aspect(
    dem, slope_rst=os.path.join(surf_folder, 'rst_slope.tif'),
    aspect_rst=os.path.join(surf_folder, 'rst_aspect.tif'),
    api="grass", slope_units="degrees",
    aspe_fromnorth=True, ws=ws
)

### Reclassify DEM, Slope and Aspect

In [ ]:
d_vars = [{
    'KEY'    : k,
    "RASTER" : vars[k],
    "RULES"  : vrules[k],
    "OUT"    : os.path.join(vars_folder, f'rcls_{os.path.basename(vars[k])}')
} for k in vars]

rcls_rasters(d_vars)

for k in vars:
    for v in d_vars:
        if k != v['KEY']:
            continue
        
        vars[k] = v['OUT']
        break

### Run Likelihood ratio

In [ ]:
vars['lulc'] = lulc_maps

firesusc = grass_lri(burn_areas, vars, dem, os.path.join(ws, 'fire_susceptibility.tif'))

## Generate Wildfire Probability

In [ ]:
fireprob = count_region_in_shape(
    burn_year, dem,
    os.path.join(ws, 'fire_probability.tif'),
    returnprob=True, nprob=nyears
)

## Produce final perigosity raster

In [ ]:
rstcalc(
    "if(fire_probability == 0, fire_probability * fire_susceptibility, 0.001 * fire_susceptibility)",
    outrst, api="grass", grids=[firesusc, fireprob]
)